In [35]:
import numpy as np

data = np.load('A09T.npz')  
signal = data['s']  # Assuming shape is (total_samples, 22)
event_types = data['etyp'].T[0]
event_positions = data['epos'].T[0]

# Initialize empty arrays
signals = []
trial_types = []
valid_labels = {769, 770, 771, 772}

for i in range(0, len(event_positions) - 1):
    event_type = event_types[i]
    next_event_type = event_types[i + 1]
    
    if event_type == 768 and next_event_type in valid_labels:  # Valid trial start
        pos = event_positions[i+1]
        
        # Extract 750 samples x 22 channels
        trial_signal = signal[pos+750 : pos+1500, 0:22]  # All 22 channels
        
        # Verify the shape is correct
        if trial_signal.shape != (750, 22):
            print(f"Unexpected shape at trial {len(signals)}: {trial_signal.shape}")
            continue
            
        signals.append(trial_signal)
        trial_types.append(next_event_type)

# Convert to numpy arrays
signals_array = np.array(signals)  # Shape (288, 750, 22)
labels_array = np.array(trial_types)  # Shape (288,)

# Verify final shapes
print("Signals shape:", signals_array.shape)
print("Labels shape:", labels_array.shape)
print("Unique labels:", np.unique(labels_array))

Unexpected shape at trial 236: (727, 22)
Signals shape: (236, 750, 22)
Labels shape: (236,)
Unique labels: [769 770 771 772]


In [36]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.nn as nn 
import torch.optim as optim 

class EEGDataset(Dataset):
    def __init__(self, trials, labels):
        # Trials: (num_trials, 750, 22)
        # Labels: (num_trials,)
        
        # Normalize data per channel
        self.data = torch.tensor(trials, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
        
        # Map original labels to 0-3
        self.label_mapping = {769: 0, 770: 1, 771: 2, 772: 3}
        self.labels = torch.tensor([self.label_mapping[int(x)] for x in labels])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Assuming you have loaded your data into trials and labels arrays
trials = np.load('eeg_signals.npy')  # Shape (288, 750, 22)
labels = np.load('eeg_labels.npy')     # Shape (288,)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    trials, labels, test_size=0.2, stratify=labels, random_state=42
)

# Create datasets and dataloaders
train_dataset = EEGDataset(X_train, y_train)
test_dataset = EEGDataset(X_test, y_test)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [37]:
import torch
import torch.nn as nn

class LTC_Cell(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(LTC_Cell, self).__init__()
        self.hidden_dim = hidden_dim
        self.W_xh = nn.Linear(input_dim, hidden_dim)
        self.W_hh = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.W_tau = nn.Linear(hidden_dim, hidden_dim)
        
    def forward(self, x, h):
        tau = torch.sigmoid(self.W_tau(h)) + 0.1
        dh = -h / tau + torch.tanh(self.W_xh(x) + self.W_hh(h))
        return h + 0.1 * dh

class LTC_Transformer(nn.Module):
    def __init__(self, input_dim=22, ltc_hidden_dim=64, num_classes=4, 
                 num_ltc_layers=1, num_transformer_layers=1, nhead=2, dropout=0.5):
        super().__init__()
        self.input_dim = input_dim
        self.ltc_hidden_dim = ltc_hidden_dim
        self.num_ltc_layers = num_ltc_layers
        
        # LTC layers
        self.ltc_layers = nn.ModuleList([
            LTC_Cell(input_dim if i==0 else ltc_hidden_dim, ltc_hidden_dim)
            for i in range(num_ltc_layers)
        ])
        
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=ltc_hidden_dim, 
            nhead=nhead, 
            dim_feedforward=ltc_hidden_dim*2,
            dropout=dropout,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(
            encoder_layer, 
            num_layers=num_transformer_layers
        )
        
        # Classifier head
        self.classifier = nn.Sequential(
            nn.Linear(ltc_hidden_dim, 32),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(32, num_classes)
        )
    
    def forward(self, x):
        # x shape: [batch_size, seq_len, input_dim]
        batch_size, seq_len, _ = x.size()
        
        # Initialize hidden states
        hiddens = [torch.zeros(batch_size, self.ltc_hidden_dim, device=x.device) 
                  for _ in range(self.num_ltc_layers)]
        
        # Process sequence through LTC layers
        all_hidden = []
        for t in range(seq_len):
            x_t = x[:, t, :]
            for layer_idx in range(self.num_ltc_layers):
                hiddens[layer_idx] = self.ltc_layers[layer_idx](
                    x_t if layer_idx == 0 else hiddens[layer_idx-1],
                    hiddens[layer_idx]
                )
            all_hidden.append(hiddens[-1])
        
        # Stack hidden states for transformer input
        hidden_stack = torch.stack(all_hidden, dim=1)  # [batch_size, seq_len, ltc_hidden_dim]
        
        # Pass through transformer
        transformer_out = self.transformer(hidden_stack)
        
        # Global average pooling
        pooled = transformer_out.mean(dim=1)
        
        # Classification
        return self.classifier(pooled)

# Example instantiation with smaller parameters
model = LTC_Transformer(
    input_dim=22,               # Number of EEG channels
    ltc_hidden_dim=64,          # LTC hidden dimension (reduced)
    num_classes=4,              # Number of classes to predict
    num_ltc_layers=1,           # Single LTC layer for simplicity
    num_transformer_layers=1,   # Single transformer layer
    nhead=2,                    # Fewer attention heads
    dropout=0.5                 # Dropout for regularization
)



In [38]:
# Initialize model
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model = LTC_Transformer(
    input_dim=22,               # Number of EEG channels
    ltc_hidden_dim=64,          # LTC hidden dimension (reduced)
    num_classes=4,              # Number of classes to predict
    num_ltc_layers=1,           # Single LTC layer for simplicity
    num_transformer_layers=1,   # Single transformer layer
    nhead=2,                    # Fewer attention heads
    dropout=0.5                 # Dropout for regularization
).to(device)

model_path = 'ltc_tran_model.pth'
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path))
    print("Loaded model from checkpoint.")
else:
    print("No checkpoint found. Starting from scratch.")

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

num_epochs = 100
previous_val_acc = None
same_acc_streak = 0
best_val_acc = 0
val_acc_list = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    
    for batch, (data, labels) in enumerate(train_loader):
        # Move data to device
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        train_loss += loss.item()
    
    # Validation
    model.eval()
    val_loss, correct, total = 0, 0, 0
   
    
    with torch.no_grad():
        for data, labels in test_loader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            val_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_acc = 100 * correct / total
    train_loss /= len(train_loader)
    val_loss /= len(test_loader)
    scheduler.step(val_loss)
    
    if previous_val_acc is None or val_acc != previous_val_acc:
        same_acc_streak = 0  # Reset counter if there's any change in accuracy
    else:
        same_acc_streak += 1  # Increment counter if accuracy is the same

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'ltc_tran_model.pth')

    val_acc_list.append(val_acc)
    
    print(f'Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | 'f'Same accuracy streak: {same_acc_streak}/10')

    # Early stopping triggered after 10 consecutive epochs with no change in accuracy
    if same_acc_streak >= 10:
        print(f'\nEarly stopping triggered after {epoch+1} epochs!')
        break
    
    # Update previous_val_acc for the next iteration
    previous_val_acc = val_acc

# Load best model and final evaluation
avg_val_acc = sum(val_acc_list) / len(val_acc_list)
print(f'\nTraining complete. Best validation accuracy: {best_val_acc:.2f}%')
print(f'Average validation accuracy over {len(val_acc_list)} epochs: {avg_val_acc:.2f}%')



C:\Users\ASUS\AppData\Local\Temp\ipykernel_26832\3717151205.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Loaded model from checkpoint.
Epoch 1/100 | Train Loss: 0.8215 | Val Loss: 1.7840 | Val Acc: 22.92% | Same accuracy streak: 0/10
Epoch 2/100 | Train Loss: 0.7391 | Val Loss: 1.7579 | Val Acc: 27.08% | Same accuracy streak: 0/10
Epoch 3/100 | Train Loss: 0.7604 | Val Loss: 1.6056 | Val Acc: 29.17% | Same accuracy streak: 0/10
Epoch 4/100 | Train Loss: 0.6875 | Val Loss: 1.8955 | Val Acc: 33.33% | Same accuracy streak: 0/10
Epoch 5/100 | Train Loss: 0.6501 | Val Loss: 1.6660 | Val Acc: 31.25% | Same accuracy streak: 0/10
Epoch 6/100 | Train Loss: 0.6343 | Val Loss: 1.9355 | Val Acc: 29.17% | Same accuracy streak: 0/10
Epoch 7/100 | Train Loss: 0.6180 | Val Loss: 1.8556 | Val Acc: 29.17% | Same accuracy streak: 1/10
Epoch 8/100 | Train Loss: 0.5654 | Val Loss: 2.0379 | Val Acc: 25.00% | Same accuracy streak: 0/10
Epoch 9/100 | Train Loss: 0.5692 | Val Loss: 1.8978 | Val Acc: 25.00% | Same accuracy streak: 1/10
Epoch 10/100 | Train Loss: 0.4869 | Val Loss: 2.0118 | Val Acc: 22.92% | Same a